In [10]:
import json
from openai import AzureOpenAI

deployment = "gpt-4o"
client = AzureOpenAI()


def get_current_cluster_state(cluster_name):
    print(f"cluster:{cluster_name}")
    return  """ERROR: Failed to pull image "chaocai/docker/dsp:latest"""

funcs = {"get_current_cluster_state": get_current_cluster_state}


def run_conversation(msg):
    response = client.chat.completions.create(
        model=deployment,
        messages=msg,
        functions=[
            {
                "name": "get_current_cluster_state",
                "description": "Get the current state in a given cluster",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "cluster_name": {
                            "type": "string",
                            "description": "the name of the cluster"
                        },
                    },
                    "required": ["cluster_name"],
                }
            }
        ],
        function_call="auto",
    )
    
    message = response.choices[0].message
    print("----- message ----")
    print(message)
    print("----- message ----")
    
    print(message.role)
    
    if not message.function_call:
        return message
    
    function_name = message.function_call.name
    function_args = json.loads(message.function_call.arguments)
    print("function_name: ", function_name, "\nfunction_args: ", function_args)
    
    res = funcs[function_name](**function_args)
    
    # message.content = None
    # msg.append(message)
    msg.append({
            "role": message.role,
            "function_call": {
                "name": function_name,
                "arguments": message.function_call.arguments,
            },
            "content": None
        })
    msg.append({
        "role": "function",
        "name": function_name,
        "content": res,
    })
    
    return run_conversation(msg)

def run(inputs):
    msg=[{"role":"user","content":inputs}]
    res = run_conversation(msg)
    return res.content



In [11]:
result = run("Hello")
print(result)

----- message ----
ChatCompletionMessage(content='Hi there! How can I assist you today?', role='assistant', function_call=None, tool_calls=None)
----- message ----
assistant
Hi there! How can I assist you today?


In [12]:
result = run("What's wrong with the cluster 'DSP'? And if there's an error, give me some suggestion. ")
print(result)

----- message ----
ChatCompletionMessage(content=None, role='assistant', function_call=FunctionCall(arguments='{"cluster_name":"DSP"}', name='get_current_cluster_state'), tool_calls=None)
----- message ----
assistant
function_name:  get_current_cluster_state 
function_args:  {'cluster_name': 'DSP'}
cluster:DSP
----- message ----
ChatCompletionMessage(content='The issue with the \'DSP\' cluster is that it failed to pull the image "chaocai/docker/dsp:latest." This could be due to several reasons, such as:\n\n1. **Incorrect Image Name or Tag:**\n   - Verify that the image name and tag are correct.\n\n2. **Network Issues:**\n   - Check the network connectivity between your cluster and the Docker registry.\n\n3. **Registry Authentication Issues:**\n   - Ensure that your cluster has the correct authentication credentials to pull images from the registry.\n\n4. **Image Not Found:**\n   - Make sure the image `chaocai/docker/dsp:latest` exists in the Docker registry.\n\n5. **Registry Downtime:*